# AFL Predictions
The file we will run to get each rounds predictions!

In [1]:
#packages
import polars as pl
import pickle
import numpy as np
import pandas as pd
#Get Update function
from ipynb.fs.defs.AFLupdate import updateData, getCurrentRoundNumbers
#Get upcoming fixture
from ipynb.fs.defs.AFLfixture import createFixture

pl.Config.set_fmt_str_lengths(150);
pl.Config.set_tbl_cols(40);
pl.Config.set_tbl_rows(50);


c:\Users\suttoe\AppData\Local\anaconda3\lib\site-packages\rpy2\robjects\lib\dplyr.py:27: UserWarning: This was designed against dplyr versions starting with 1.0 but you have 1.1.4
  warnings.warn(


In [2]:
#Run updates
updateData()

i Returning data for "All Rounds, 2025"
v Returning data for "All Rounds, 2025" s]

No update Required


In [3]:
#get the fixture 
fixture, predictionDF = createFixture()

i Returning data for "All Rounds, 2025"
v Returning data for "All Rounds, 2025" s]

i Returning data for "All Rounds, 2025"
v Returning data for "All Rounds, 2025" s]

i Returning data for "Round 28, 2024"
v Returning data for "Round 28, 2024" s]

i Returning data for "All Rounds, 2025"
v Returning data for "All Rounds, 2025" s]

i Returning data for "All Rounds, 2025"
v Returning data for "All Rounds, 2025" s]

i Returning data for "All Rounds, 2025"
v Returning data for "All Rounds, 2025" s]

i Fetching match ids
v Fetching match ids s]

i Finding player stats for 216 matches.
v Finding player stats for 216 matches. [45.6s]

Hi


In [4]:
#if fixture exists i.e no issues have been introduced, go ahead and use our models to get our predictions
modelWin = pickle.load(open('Data/winner.sav', 'rb'))

predictWinner = modelWin.predict(fixture).tolist()
probsWinner = modelWin.predict_proba(fixture).tolist()

#create the winner portion of the prediction df
predictionDF = (predictionDF
                 #place in results
                 .with_columns(pl.Series(predictWinner).alias('Predictions'),
                               pl.Series(probsWinner).alias('Probs'))
                 #Winner probs
                 .with_columns(((pl.col('Probs').list.get(pl.col('Predictions')))*100).round(2).alias('Win Prob'))
                 .with_columns(pl.when(pl.col('Predictions') == 1).then(pl.col('home.team.name')).otherwise(pl.col('away.team.name')).alias('Predicted Winner'))
                 #Add current round
                 .with_columns(pl.lit(getCurrentRoundNumbers()).alias('Round Number'))
                 #rename
                 .rename({'home.team.name' : 'Home Team',
                          'away.team.name' : 'Away Team'})
                 #keep only wants required
                 .select('Round Number', 'Home Team', 'Away Team', 'Predicted Winner', 'Win Prob')
                )


i Returning data for "All Rounds, 2025"
v Returning data for "All Rounds, 2025" s]



In [17]:
#Get the margin
#load the model in
model = pickle.load(open('Data/margin.sav', 'rb'))

#And the win probablities to fixture
fixture[['homeProb', 'awayProb']] = probsWinner
#predict the fixture
predictMargin = model.predict(fixture).tolist()

#Add to prediction df
predictionDF = (predictionDF.with_columns(pl.Series(predictMargin).round(0).cast(pl.Int32).alias('Margin')))


In [21]:
predictionDF

Round Number,Home Team,Away Team,Predicted Winner,Win Prob,Margin
i32,str,str,str,f64,i32
0,"""Brisbane Lions""","""Geelong Cats""","""Brisbane Lions""",94.65,35
0,"""Sydney Swans""","""Hawthorn""","""Sydney Swans""",75.5,32
0,"""Gold Coast Suns""","""Essendon""","""Gold Coast Suns""",89.22,24
0,"""GWS Giants""","""Collingwood""","""GWS Giants""",81.49,28


In [20]:
#Write to folder
predictionDF.write_csv(fr'Tips/2025/Round_{getCurrentRoundNumbers()}.csv')

i Returning data for "All Rounds, 2025"
v Returning data for "All Rounds, 2025" s]

